In [ ]:
from ..EA.population_elitist import PopulationElitist
from numpy.random import rand, choice
from copy import deepcopy

class PopulationNSGA2(PopulationElitist):

    def __init__(self, crossover_probability, crossover_index, mutation_probability, mutation_index):
        super().__init__()
        self.crossover_probability = crossover_probability
        self.crossover_index = crossover_index
        self.mutation_probability = mutation_probability
        self.mutation_index = mutation_index

    def initialize(self, problem, N):
        for i in range(N):
            self.individuals.append(problem.generate_random_nsga2_individual())
        self.N = N

    def merge(self, other):
        new = PopulationNSGA2(self.crossover_probability, self.crossover_index, self.mutation_probability, self.mutation_index)
        new.append_many(self.individuals)
        new.append_many(other.individuals)
        return new

    def removeLast(self):
        del self.individuals[-1]
        self.N -= 1

    def simulatedBinaryCrossover(self, p1, p2):
        coordinates1 = []
        coordinates2 = []
        eta = self.crossover_index + 1
        epsilon = 10**(-12)
        for j in range(p1.problem.d):
            if p1.coordinates[j] < p2.coordinates[j]:
                y1 = p1.coordinates[j]
                y2 = p2.coordinates[j]
            else:
                y1 = p2.coordinates[j]
                y2 = p1.coordinates[j]
            if y2-y1 > epsilon:
                beta = 1 + 2/(y2-y1) * min(y1-p1.problem.boundaries[j][0], p1.problem.boundaries[j][1]-y2)
                alpha = 2 - beta**(-eta)
                u = rand()
                if u <= 1/alpha:
                    gamma = (u*alpha)**(1/eta)
                else:
                    gamma = (1/(2-u*alpha))**(1/eta)
            else:
                gamma = 1
            coordinates1.append( 0.5*( y1 + y2 - gamma*(y2-y1)) )
            coordinates2.append( 0.5*( y1 + y2 + gamma*(y2-y1)) )
        c1 = p1.problem.generate_nsga2_individual(coordinates1)
        c2 = p2.problem.generate_nsga2_individual(coordinates2)
        return c1, c2

    def binaryTournamentSelection(self, p1, p2):
        if p1.crowdedComparison(p2):
            p = deepcopy(p1)
        else:
            p = deepcopy(p2)
        return p

    def offspringGeneration(self):
        self.fastNondominatedSort()
        self.crowdingDistanceSort()
        Q = PopulationNSGA2(self.crossover_probability, self.crossover_index, self.mutation_probability, self.mutation_index)
        while len(Q) < self.N:
            p1, p2 = choice(self.individuals, 2, replace=True)
            if rand() < self.crossover_probability:
                Q.append_many(self.simulatedBinaryCrossover(p1, p2))
            else:
                Q.append(self.binaryTournamentSelection(p1, p2))
        if len(Q) > self.N:
            Q.removeLast()
        for q in Q.individuals:
            if rand() < self.mutation_probability:
                q.polynomialMutation(self.mutation_index)
        return Q

    
        





